### Strat

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import ipywidgets as widgets
import plotly.graph_objects as go
import seaborn as sns
sns.set()

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import random
import cv2
from sklearn import mixture
from sklearn.cluster import KMeans

df = pd.read_csv('tuning_param_all_cnn8.csv')

In [2]:
def make_gaussian(g_size, p):
    gaussian = np.zeros((g_size, g_size))

    rotation_mat = np.array([[np.cos(p[5]), -np.sin(p[5])], [np.sin(p[5]), np.cos(p[5])]])
    center_vert = np.array([p[1], p[2]]).reshape((-1, 1))

    for x in range(g_size):
        for y in range(g_size):
            point = np.array([x, y]).reshape((-1, 1))
            transrated_point = np.dot(rotation_mat, (point - center_vert))

            gaussian[x, y] = p[0] * np.exp((-1 / 2) * ((transrated_point[0] / p[3]) ** 2 + (transrated_point[1] / p[4]) ** 2))

    return gaussian


def make_gaussians2_sum(num_gaus, g_size, p):
    gaussian = np.zeros((g_size, g_size))

    for n in range(num_gaus):
        gaussian += make_gaussian(g_size, p[n * 6: (n + 1) * 6])

    return gaussian


def fine_tuning(g_size, p, scale, theta_all, trans_x, trans_y):
    k = p.copy()
    x_mid = (k[1] + k[7]) / 2
    y_mid = (k[2] + k[8]) / 2
    x1_new = np.array(k[1] - x_mid) * np.cos(theta_all) - np.array(k[2] - y_mid) * np.sin(theta_all) + x_mid
    y1_new = np.array(k[1] - x_mid) * np.sin(theta_all) + np.array(k[2] - y_mid) * np.cos(theta_all) + y_mid
    x2_new = np.array(k[7] - x_mid) * np.cos(theta_all) - np.array(k[8] - y_mid) * np.sin(theta_all) + x_mid
    y2_new = np.array(k[7] - x_mid) * np.sin(theta_all) + np.array(k[8] - y_mid) * np.cos(theta_all) + y_mid
    k[1] = x1_new
    k[2] = y1_new
    k[7] = x2_new
    k[8] = y2_new
    k[1] += g_size * (scale - 1) / 2
    k[2] += g_size * (scale - 1) / 2
    k[7] += g_size * (scale - 1) / 2
    k[8] += g_size * (scale - 1) / 2
    k[1] += trans_x * scale
    k[2] += trans_y * scale
    k[7] += trans_x * scale
    k[8] += trans_y * scale
    k[5] -= theta_all
    k[11] -= theta_all
    return k


def make_tuned_gaussians(g_size, p, x):
    if x[0] != 1.8:
        return np.ones((g_size, g_size))
    param_new = fine_tuning(g_size, p, x[0], x[1], x[2], x[3])
    # print("x:", x)
    new_size = int(g_size * x[0])
    tuned_img = make_gaussians2_sum(2, new_size, param_new)
    tuned_gaussian = cv2.resize(tuned_img, (g_size, g_size), interpolation=cv2.INTER_AREA)
    return tuned_gaussian


def plot(unit_use):

    tgt_layer = 'convolution_8'
    u_num = 512

    data_all = np.zeros((u_num, 128, 128, 3))
    data_black = np.zeros((128, 128, 3))
    for i in range(u_num):
        data_all[i] = pd.read_pickle("./visualize_laplacian/8-15/visualize-lap-{}-{}-15.pkl".format(tgt_layer, i))[64:192, 64:192]

    data_row = data_all[unit_use]
    # print("data_row:\n", data_row)
    data_row = np.clip(data_row, 0., 1.)


    ##################### ↓↓↓ feature map preprocessing↓↓↓ #####################

    # Use Gaussian blur to remove some noise
    data_blur = cv2.GaussianBlur(data_row, (3, 3), 0)

    # Find the central region
    image_array = data_blur.reshape((data_blur.shape[0] * data_blur.shape[1], 3))

    # 1st: GMM cluster
    clst = mixture.GaussianMixture(n_components=3, max_iter=100, covariance_type="full")
    clst.fit(image_array)
    predicted_labels = clst.predict(image_array)
    centroids_GMM = clst.means_
    # print("centroids_GMM", centroids_GMM)
    labels = predicted_labels
    label_matrix = labels.reshape((128, 128))

    baseline = np.array([0.5] * 9).reshape(3, 3)

    max_index = np.argmax(np.sum(abs(centroids_GMM - baseline), axis=1))
    center_mean = centroids_GMM[max_index].dot(np.array([0.2989, 0.5870, 0.1140]).T)
    # print("center_mean:", center_mean)

    data_filled = data_blur.dot(np.array([0.2989, 0.5870, 0.1140]).T) - center_mean

    data_center = data_blur.copy()
    if max_index == 0:
        for i in range(128):
            for j in range(128):
                if label_matrix[i, j] != 0:
                    data_center[i][j] = [center_mean, center_mean, center_mean]

    elif max_index == 1:
        for i in range(128):
            for j in range(128):
                if label_matrix[i, j] != 1:
                    data_center[i][j] = [center_mean, center_mean, center_mean]
    else:
        for i in range(128):
            for j in range(128):
                if label_matrix[i, j] != 2:
                    data_center[i][j] = [center_mean, center_mean, center_mean]

    data_filled_center = data_center.dot(np.array([0.2989, 0.5870, 0.1140]).T) - center_mean

    
    image_array_new = data_center.reshape((data_row.shape[0] * data_row.shape[1], 3))

    # 2nd: k-means cluster
    kmeans = KMeans(n_clusters=3,  init='k-means++')
    kmeans.fit(image_array_new)
    centroids_kmeans = kmeans.cluster_centers_
    labels = kmeans.labels_
    # print("centroids_k-means", centroids_kmeans)
    label_matrix_new = labels.reshape((128, 128))

    data_0_new = data_center.copy()
    data_1_new = data_center.copy()
    data_2_new = data_center.copy()
    for i in range(128):
        for j in range(128):
            if label_matrix_new[i, j] == 0:
                data_0_new[i][j] = [1, 1, 1]
            else:
                data_0_new[i][j] = [0, 0, 0]
            if label_matrix_new[i, j] == 1:
                data_1_new[i][j] = [1, 1, 1]
            else:
                data_1_new[i][j] = [0, 0, 0]
            if label_matrix_new[i, j] == 2:
                data_2_new[i][j] = [1, 1, 1]
            else:
                data_2_new[i][j] = [0, 0, 0]


    # Determine the facilitative and suppressive region
    centroids_filled = centroids_kmeans.dot(np.array([0.2989, 0.5870, 0.1140]).T)
    pos_index = np.argmax(centroids_filled)
    neg_index = np.argmin(centroids_filled)
    # print("pos_index, neg_index = ", pos_index, neg_index)
    data_pos = data_0_new if pos_index == 0 else (data_1_new if pos_index == 1 else data_2_new)
    data_neg = data_0_new if neg_index == 0 else (data_1_new if neg_index == 1 else data_2_new)


    # Erosion and dilation
    kernel = np.ones((4, 4), np.uint8)
    data_pos = cv2.morphologyEx(cv2.morphologyEx(data_pos, cv2.MORPH_CLOSE, kernel), cv2.MORPH_OPEN, kernel)
    data_neg = cv2.morphologyEx(cv2.morphologyEx(data_neg, cv2.MORPH_CLOSE, kernel), cv2.MORPH_OPEN, kernel)

    data_mod = data_filled_center.copy()
    for i in range(128):
        for j in range(128):
            if (data_pos[i, j] == [1, 1, 1]).all() and (data_neg[i, j] != [1, 1, 1]).all():
                data_mod[i][j] = abs(data_mod[i][j])
            elif (data_neg[i, j] == [1, 1, 1]).all() and (data_pos[i, j] != [1, 1, 1]).all():
                data_mod[i][j] = -abs(data_mod[i][j])
            elif (data_pos[i, j] == [1, 1, 1]).all() and (data_neg[i, j] == [1, 1, 1]).all():
                data_mod[i][j] = data_mod[i][j] / 2

    # Average blur
    data_mod = cv2.blur(data_mod, (2, 2))
    score_plane = np.sqrt(np.sum(data_mod ** 2))

    ################# PLOT ##################

    plt.figure(figsize=(10,10))

    # 150：450
    param_init = [0.4356, 78.61, 64.08, 19.72, 12.39, 1.477,
                  -0.38804, 48.145, 64.923, 32, 32, 0.089]

    p = df.loc[unit_use].values.tolist()[2:]

    plt.subplot(131)
    plt.imshow(data_row)

    p_tuned = fine_tuning(128, param_init, p[0], p[1], p[2], p[3])

    new_size = int(128 * p[0])
    tuned_img = make_gaussians2_sum(2, new_size, p_tuned)
    tuned_img_norm = cv2.resize(tuned_img, (128, 128), interpolation=cv2.INTER_AREA)

    plt.subplot(132)
    plt.imshow(data_mod)

    plt.subplot(133)
    plt.imshow(tuned_img_norm)
    
    score_diff = np.sqrt(np.sum((data_mod - tuned_img_norm) ** 2))
    print("score_diff:", score_diff)

    score_mod = np.sqrt(np.sum(data_mod ** 2))
    print("score_mod:", score_mod)

    score_gaussian = np.sqrt(np.sum(tuned_img_norm ** 2))
    print("score_gaussian:", score_gaussian)

    pro1 = score_diff / score_gaussian
    print("score_diff / score_gaussian = ", pro1)

    pro2 = score_diff / score_mod
    print("score_diff / score_mod = ", pro2)

    plt.show()

### Plot

In [3]:
# interactive_plot = widgets.interactive(plot, unit_use=(0, 511))
# interactive_plot

slider = widgets.IntSlider(min=0, max=511, value=0)
btn = widgets.Button(description="Show")

def btn_click(sender):
    plot(slider.value)

btn.on_click(btn_click)
display(slider, btn)

IntSlider(value=0, max=511)

Button(description='Show', style=ButtonStyle())

In [ ]:
sns.jointplot(x='pro1', y='pro2', data=df, kind="reg")
plt.show()

In [ ]:
fig = go.Figure(data=go.Scatter(
    x=df["pro1"],
    y=df["pro2"],
    mode="markers",
    marker=dict(
        size=8,
        color=df["score"],
        colorscale="Viridis",
        showscale=True
    ),
    text=df.index),
    layout = dict(title="Result", xaxis=dict(title="score_diff / score_gaussian"), yaxis=dict(title="score_diff / score_mod"))
)
fig.show()

$$
score\_diff = \sqrt{\sum^{127}_{i,j=0}({data\_mod}_{ij}-data\_gau_{ij})^2}
$$

$$
score\_mod = \sqrt{\sum^{127}_{i,j=0}({data\_mod}_{ij})^2}
$$

$$
score\_gau = \sqrt{\sum^{127}_{i,j=0}({data\_gau}_{ij})^2}
$$